In [ ]:
#this file has APIS for 4 different purpose, these can be used to automate our tasks
#within each API there are more functions that can be accessed using apporopriate routing provided

In [5]:
from flask import Flask, jsonify
import pandas as pd



In [6]:
data= pd.read_csv('dirty_songs.csv')

C:\Users\Dell\AppData\Local\Temp\ipykernel_15116\469756692.py:1: DtypeWarning: Columns (1,4,7,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv('songs.csv')


In [7]:
#removing dirty data API


app = Flask(__name__)

@app.route('/')
def index():
    return 'hello we will be removing dirty data through this'
# Route to detect and remove potential data entry errors
@app.route('/detect_errors')
def detect_and_remove_errors(file_path):
    # Initialize a list to store potential errors
    errors = []

    # Check for negative values in numeric columns
    numeric_columns = data.select_dtypes(include=['float', 'int']).columns
    for column_name in numeric_columns:
        negative_values = data[column_name] < 0
        if negative_values.any():
            error = f"Column '{column_name}' has {negative_values.sum()} negative value(s)"
            errors.append(error)

    # Check for text values that are too long
    text_columns = data.select_dtypes(include=['object']).columns
    for column_name in text_columns:
        if data[column_name].str.len().max() > 100:
            error = f"Column '{column_name}' has text values that are too long"
            errors.append(error)

    # Remove duplicate rows
    data.drop_duplicates(inplace=True)

    # Remove columns with too many repeated values
    for column_name in data.columns:
        if data[column_name].nunique() < len(data) / 10:
            data.drop(columns=[column_name], inplace=True)

    # Return the potential errors and cleaned data as a JSON response
    if errors:
        return jsonify({'errors': errors, 'data': data.to_dict()})
    else:
        return jsonify({'message': 'No potential errors detected', 'data': data.to_dict()})

if __name__ == '__main__':
    app.run(port=6500)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:6500
Press CTRL+C to quit


In [ ]:
#API for analysing missing values
from flask import Flask, jsonify
import pandas as pd

app = Flask(__name__)

# Read the data from a CSV file
@app.route('/')
def index():
    return 'hello we will be analysis missing values'
# Route to perform missing value analysis on a dataset
@app.route('/missing_analysis')
def missing_value_analysis():
    # Calculate the percentage of missing values for each column
    missing_percentage = round(data.isnull().sum() / len(data) * 100, 2)

    # Identify columns with missing values
    missing_columns = data.columns[data.isnull().any()].tolist()

    # Create a dictionary to store the missing value analysis results
    results = {
        'missing_percentage': missing_percentage.to_dict(),
        'missing_columns': missing_columns,
    }

    # If any columns have missing values, also perform imputation
    if missing_columns:
        # Identify the data types of the columns with missing values
        missing_column_types = data[missing_columns].dtypes

        # Impute missing values for numeric columns with the column mean
        numeric_columns = missing_column_types[missing_column_types != 'object'].index.tolist()
        if numeric_columns:
            imputed_data = data.copy()
            for column_name in numeric_columns:
                mean = imputed_data[column_name].mean()
                imputed_data[column_name].fillna(mean, inplace=True)
            results['imputed_data'] = imputed_data.head(10).to_dict(orient='records')

        # Impute missing values for text columns with the mode
        text_columns = missing_column_types[missing_column_types == 'object'].index.tolist()
        if text_columns:
            imputed_data = data.copy()
            for column_name in text_columns:
                mode = imputed_data[column_name].mode()[0]
                imputed_data[column_name].fillna(mode, inplace=True)
            results['imputed_data'] = imputed_data.head(10).to_dict(orient='records')

    # Return the missing value analysis results as a JSON response
    return jsonify(results)

if __name__ == '__main__':
    app.run(port=7000) 


In [ ]:
#APIS for statistical tests:
# Import libraries
from flask import Flask, jsonify, request
import pandas as pd
import scipy.stats as stats
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt


# Create a Flask app object
app = Flask(__name__)

@app.route('/')
def index():
    return "Hello you can add your staistical tests routes to generate specific tests"

# Define routes using the @app decorator

@app.route('/boxplot')
def boxplot():
    column_name = 'acousticness' #this can be modified according to your choice
    column_data = data[column_name]

    # Ignore missing values
    column_data = column_data[~column_data.isna()]

    # Generate the boxplot
    fig, ax = plt.subplots()
    ax.boxplot(column_data)
    ax.set_title(f"Boxplot of {column_name}")
    ax.set_xlabel(column_name)
    ax.set_ylabel("Value")

    # Save the plot to a file and return the file path
    plot_path = 'boxplot.png' #it will be saved in your downloads, and can simply be seen when you quit from API running
    plt.savefig(plot_path)

    return jsonify({'plot_path': plot_path})


@app.route('/histogram')
def histogram():
    column_name = 'acousticness'
    column_data = data[column_name]

    # Generate the histogram
    fig, ax = plt.subplots()
    ax.hist(column_data.dropna(), bins=10)
    ax.set_title(f"Histogram of {column_name}")
    ax.set_xlabel(column_name)
    ax.set_ylabel("Frequency")

    # Save the plot to a file and return the file path
    plot_path = 'histogram.png'
    plt.savefig(plot_path)

    return jsonify({'plot_path': plot_path})

@app.route('/correlation')
def correlation():
    column_name = 'acousticness'
    
    # Get the correlation matrix for the specified column
    corr = data.corr()[[column_name]]
    
    # Generate the heatmap
    fig, ax = plt.subplots()
    sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, ax=ax)
    ax.set_title(f"Correlation Matrix for {column_name}")
    
    # Save the plot to a file and return the file path
    plot_path = 'correlation.png'
    plt.savefig(plot_path)

    return jsonify({'plot_path': plot_path})

@app.route('/regression')
def regression():
    x_col = 'energy'
    y_col = 'tempo'

    # Generate the regression plot
    sns.regplot(data=data, x=x_col, y=y_col, dropna=False)

    # Save the plot to a file and return the file path
    plot_path = 'regplot.png'
    plt.savefig(plot_path)

    return jsonify({'plot_path': plot_path})


@app.route('/chisquare')
def chisquare():
    column1_name = 'energy'
    column2_name = 'danceability'
    contingency_table = pd.crosstab(data[column1_name], data[column2_name])
    
    # Perform chi-square test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)

    # Return the results as JSON response
    result = {
        'test': 'Chi-Square Test',
        'variables': f'{column1_name} vs {column2_name}',
        'chi2_statistic': chi2,
        'p_value': p_value,
        'degrees_of_freedom': dof,
        'expected_values': expected.tolist()
    }
    return jsonify(result)

@app.route('/anova')
def anova():
    # Read the data from the CSV file
    data = pd.read_csv('path/to/file.csv')

    # Extract the data for the two columns and ignore missing values
    column1_name = 'energy'
    column2_name = 'tempo'
    column1_data = data[column1_name][~data[column1_name].isnull()]
    column2_data = data[column2_name][~data[column2_name].isnull()]

    # Perform the ANOVA test
    f_value, p_value = stats.f_oneway(column1_data, column2_data)

    # Print the results of the test
    if p_value < 0.05:
        result = "The difference between the means of the two columns is statistically significant (p-value < 0.05)"
    else:
        result = "There is no statistically significant difference between the means of the two columns (p-value >= 0.05)"

    # Return the results as a JSON response
    return jsonify({'result': result})

if __name__ == '__main__':
    app.run(port=7500)

In [ ]:
#API to directly identify outliers and remove the ones that are over an extreme range
from flask import Flask, jsonify

app = Flask(__name__)

# Route to detect outliers in a given column
@app.route('/')
def index():
    return "hello, we will be detecting outliers and removing extreme ones here"

@app.route('/detect_outliers')
def detect_outliers():
    column_name = 'acousticness'
    iqr_factor = 1.5
    column_data = raw[column_name]

    # Calculate the IQR of the column data
    q1 = column_data.quantile(0.25)
    q3 = column_data.quantile(0.75)
    iqr = q3 - q1

    # Define the upper and lower bounds for outlier detection
    # Define the upper and lower bounds for outlier detection
    upper_bound = q3 + iqr_factor * (3 * iqr)
    lower_bound = q1 - iqr_factor * (3 * iqr)

    # Find the outliers in the column data
    outliers = column_data[(column_data > upper_bound) | (column_data < lower_bound)]

    # Remove the outliers that are even 1.5 times outside the range
    remove_outliers = outliers[(outliers > upper_bound * 1.5) | (outliers < lower_bound * 1.5)]
    column_data_cleaned = column_data[~column_data.isin(remove_outliers)]

    # Convert the outliers to a list and return as JSON
    outliers_list = remove_outliers.tolist()
    return jsonify({'outliers': outliers_list, 'column_data_cleaned': column_data_cleaned.tolist()})

if __name__ == '__main__':
    app.run(port=8000)